# 📘 Para-Procesimi i të Dhënave për Analizë
## Tema: Analiza e të dhënave të shitjeve (Superstore Dataset)

Ky notebook përmban hapat e detajuar të para-procesimit për dataset-in **Superstore**.
Procesi përfshin:
- Ngarkimin dhe inspektimin e të dhënave
- Trajtimin e tipeve të dhënave dhe vlerave të zbrazëta
- Pastrimin, agregimin dhe mostrimin
- Diskretizimin, binarizimin dhe transformimet
- Reduktimin e dimensioneve me PCA

**👩‍🎓 Studentja:** Anjeza Sfishta  <br>
**📚 Lënda:** Përgatitja dhe vizualizimi i të dhënave  

##  Importimi i librarive
Në këtë seksion importojmë libraritë kryesore që do të përdorim gjatë para-procesimit.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, Binarizer, KBinsDiscretizer
from sklearn.decomposition import PCA
plt.rcParams['font.family'] = 'DejaVu Sans'  # Font i paracaktuar dhe Unicode-kompatibil
from scipy import stats

print('Pandas version:', pd.__version__)
print('Seaborn version:', sns.__version__)

## Ngarkimi i të dhënave
Lexojmë dataset-in dhe shikojmë rreshtat e parë për të kuptuar strukturën e tij.


In [ ]:
df = pd.read_csv('superstore.csv')
df.head()

##  Inspektimi dhe cilësia e të dhënave
Analizojmë tipet, mungesat dhe statistikat për të vlerësuar cilësinë e dataset-it.


In [ ]:
df.info()
df.describe()
df.isnull().sum()

##  Pastrimi i të dhënave
Heqim duplikatet dhe rreshtat me vlera të zbrazëta, si dhe konvertojmë kolonat e datës në formatin e duhur.


In [ ]:
df = df.drop_duplicates()
#df = df.dropna()
for col in ['Sales']:
    df[col] = df[col].fillna(df[col].median())
    
df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')
df['Ship Date'] = pd.to_datetime(df['Ship Date'], errors='coerce')

invalid_dates = df[df['Order Date'].isna() | df['Ship Date'].isna()]
print(f"Rreshta me data të pavlefshme: {len(invalid_dates)}")



In [ ]:
plt.figure(figsize=(10,5))
sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
plt.title('Vizualizimi i vlerave të zbrazëta')
plt.show()

df[['Sales']].hist(bins=30, figsize=(6,4))
plt.title('Shpërndarja e Sales')
plt.show()

In [ ]:
# Heq rreshtat ku datat janë NaT
df = df.dropna(subset=['Order Date', 'Ship Date'])
print(f"Numri i rreshtave pas heqjes së datave të pavlefshme: {len(df)}")


print("✅ Të dhënat u pastruan dhe datat u konvertuan me sukses!")

##  Diskretizimi, Binarizimi dhe Standardizimi
Kryejmë:
- **Diskretizimin** e kolonës `Sales` në 4 intervale  
- **Binarizimin** e kolonës `Sales` për të ndarë fitimet nga humbjet  
- **Standardizimin** e kolonave numerike për t’i sjellë në një shkallë të përbashkët


In [ ]:
# Diskretizimi i kolonës 'Sales' në 4 intervale
disc = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='quantile')
df['Sales_binned'] = disc.fit_transform(df[['Sales']])
print("Kufijtë e intervaleve për Sales:", disc.bin_edges_)


# Binarizimi i kolonës 'Sales' sipas mesatares (1 = Shitje e lartë, 0 = Shitje e ulët)
threshold = df['Sales'].mean()
binarizer = Binarizer(threshold=threshold)
df['Sales_binary'] = binarizer.fit_transform(df[['Sales']])

# Standardizimi i kolonave numerike
scaler = StandardScaler()
df['Sales_original'] = df['Sales']  # ruaj vlerat para standardizimit
df[['Sales']] = scaler.fit_transform(df[['Sales']])

print("✅ Diskretizimi, binarizimi dhe standardizimi u kryen me sukses!")
df[['Sales', 'Sales_binned', 'Sales_binary']].head()


##  Correlation Matrix për kolonat numerike


In [ ]:
# Gjej kolonat numerike automatikisht
num_cols = df.select_dtypes(include=['number']).columns

plt.figure(figsize=(6,5))
sns.heatmap(df[num_cols].corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()


##  Reduktimi i dimensioneve me PCA
Përdorim **PCA (Principal Component Analysis)** për të reduktuar dimensionet e të dhënave dhe për të vizualizuar strukturën e tyre.


In [ ]:
pca = PCA(n_components=1)
pca_data = pca.fit_transform(df[['Sales']])
df['PCA1'] = pca_data[:, 0]

plt.figure(figsize=(10,6))
sns.histplot(df['PCA1'], kde=True, color='skyblue', alpha=0.6)
plt.title('Vizualizimi i komponentit të parë PCA (Sales)')
plt.xlabel('PCA1')
plt.ylabel('Shpërndarja')
plt.show()

explained_variance = pca.explained_variance_ratio_[0]
print(f"Varianca e kapur nga PCA1: {explained_variance:.2f}")

##  Krijimi i veçorive të reja
Shtohet veçoria e re `Delivery_Days` që tregon kohën e dërgesës për çdo porosi.


In [ ]:
df['Delivery_Days'] = (df['Ship Date'] - df['Order Date']).dt.days
df[['Order Date', 'Ship Date', 'Delivery_Days']].head()

##  Mostrim dhe Agregim i të Dhënave

Në këtë seksion do të kryejmë disa analiza përmbledhëse duke përdorur funksione të tilla si:
- `groupby()` për të grupuar të dhënat sipas kategorive ose rajoneve
- `agg()` për të llogaritur mesatare, total dhe numërime
- `sample()` për të marrë mostra të rastësishme nga dataset-i

Këto analiza ndihmojnë për të kuptuar shpërndarjen e shitjeve dhe fitimeve sipas rajonit, kategorisë ose segmentit të klientit.


 Shembuj Praktikë

1. Mostrim i rastësishëm (10% e dataset-it)

In [ ]:
sample_size = int(0.1 * len(df))  # 10% e dataset-it
print(f"Mostër e rastësishme (10% e dataset-it): {sample_size} rreshta")
display(df.sample(sample_size))



2. Mesatarja e shitjeve dhe fitimit sipas Rajonit

In [ ]:
print("\nMesatarja e shitjeve dhe fitimit sipas Rajonit:")
region_summary = df.groupby('Region')[['Sales']].mean().reset_index()
display(region_summary)

3. Totali i shitjeve dhe fitimit sipas Kategorisë së Produktit

In [ ]:
print("\nTotali i shitjeve dhe fitimit sipas Kategorisë së Produktit:")
category_summary = df.groupby('Category')[['Sales',]].sum().reset_index()
display(category_summary)

4. Numri i porosive sipas Segmentit të Klientit

In [ ]:
print("\nNumri i porosive sipas Segmentit të Klientit:")
segment_orders = df['Segment'].value_counts().reset_index()
segment_orders.columns = ['Segment', 'Numri i Porosive']
display(segment_orders)

5. Vizualizim grafik i shitjeve mesatare sipas Rajonit

In [ ]:
plt.figure(figsize=(8,5))
sns.barplot(data=region_summary, x='Region', y='Sales')
plt.title(" Mesatarja e Shitjeve sipas Rajonit", fontsize=13)
plt.xlabel("Rajoni")
plt.ylabel("Mesatarja e Shitjeve")
plt.show()

##  Analiza shtesë e shitjeve sipas kategorive, dërgesave dhe kohës


In [ ]:
# 📊 Shitjet sipas kategorisë
plt.figure(figsize=(8,5))
sns.barplot(data=category_summary, x='Category', y='Sales', hue='Category', palette='pastel', legend=False)
plt.title('Totali i shitjeve sipas Kategorisë', fontsize=13)
plt.xlabel('Kategoria')
plt.ylabel('Totali i Shitjeve')
plt.show()

# 📦 Shitjet sipas mënyrës së dërgimit
shipmode_sales = df.groupby('Ship Mode')['Sales'].mean().reset_index()
plt.figure(figsize=(8,5))
sns.barplot(data=shipmode_sales, x='Ship Mode', y='Sales', hue='Ship Mode', palette='muted', legend=False)
plt.title('Mesatarja e shitjeve sipas mënyrës së dërgimit', fontsize=13)
plt.xlabel('Mënyra e Dërgimit')
plt.ylabel('Mesatarja e Shitjeve')
plt.show()

# 📆 Shto kolonat për muajin dhe vitin e porosisë
df['Order_Month'] = df['Order Date'].dt.month
df['Order_Year'] = df['Order Date'].dt.year

# 📈 Shitjet sipas muajit
monthly_sales = df.groupby('Order_Month')['Sales'].sum().reset_index()
plt.figure(figsize=(8,5))
sns.lineplot(data=monthly_sales, x='Order_Month', y='Sales', marker='o', color='orange')
plt.title('Shitjet mujore totale', fontsize=13)
plt.xlabel('Muaji')
plt.ylabel('Totali i Shitjeve')
plt.xticks(range(1,13))
plt.show()


# Pjesa II : Detektimi i Përjashtuesve dhe Eksplorimi i të Dhënave

In [ ]:
# Ngarkojmë dataset-in pas pastrimit nga Pjesa I
df = pd.read_csv("cleaned_superstore.csv")

# Kolonat numerike për analizë
num_cols = ['Sales', 'Delivery_Days']

### Statistika përmbledhëse

In [ ]:
print(" Statistika përmbledhëse për kolonat numerike:")
display(df[num_cols].describe())


### Detektimi i outlier-ëve

1) Me metodën IQR

In [ ]:
Q1 = df[num_cols].quantile(0.25)
Q3 = df[num_cols].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers_iqr = df[((df[num_cols] < lower_bound) | (df[num_cols] > upper_bound)).any(axis=1)]
print(f"Numri i rreshtave me outliers (IQR): {len(outliers_iqr)}")
display(outliers_iqr.head())


2) Me Z-scores

In [ ]:
z_scores = np.abs(stats.zscore(df[num_cols]))
outliers_z = df[(z_scores > 3).any(axis=1)]
print(f"Numri i rreshtave me outliers (Z-score): {len(outliers_z)}")
display(outliers_z.head())


3) Me Grubbs’ test (për kolonën 'Sales')

In [ ]:
def grubbs_test(series, alpha=0.05):
    """
    Kthen indeksin e vlerës më ekstreme sipas Grubbs' test.
    Kjo është për një vlerë ekstreme njëherësh.
    """
    n = len(series)
    mean_y = np.mean(series)
    std_y = np.std(series, ddof=1)
    
    # Llogarit devijimin absolut nga mesatarja
    abs_diff = np.abs(series - mean_y)
    max_deviation_index = abs_diff.argmax()
    
    # Vlera e testit G
    G_calculated = abs_diff[max_deviation_index] / std_y
    
    # G kritik nga formula e Grubbs
    t_critical = stats.t.ppf(1 - alpha / (2*n), n - 2)
    G_critical = ((n - 1)/np.sqrt(n)) * np.sqrt(t_critical**2 / (n - 2 + t_critical**2))
    
    # Kontrolli nëse është outlier
    if G_calculated > G_critical:
        return max_deviation_index
    else:
        return None

# Përdorimi
index_outlier = grubbs_test(df['Sales'], alpha=0.05)
if index_outlier is not None:
    print("Indeksi i outlier-it sipas Grubbs’ test:", index_outlier)
    display(df.loc[[index_outlier]])
else:
    print("Nuk ka outlier të konstatueshëm sipas Grubbs’ test.")


### Vizualizime për outlier-ët

In [ ]:
# Boxplot për kolonat numerike
plt.figure(figsize=(8,5))
sns.boxplot(data=df[num_cols])
plt.title('Detektimi i përjashtuesve - Boxplot')
plt.show()

# Scatter plot Sales vs Delivery_Days
plt.figure(figsize=(8,5))
sns.scatterplot(x='Delivery_Days', y='Sales', data=df)
plt.title('Shitjet ndaj Delivery_Days')
plt.show()
